In [1]:
%matplotlib inline

In [2]:
from datetime import datetime

In [3]:
#renderer
from PIL import ImageFont
import numpy as np
import cv2
import torch.nn as nn
from torch.nn.utils import clip_grad_norm_

char_size = 24
# char render
def render(text, font=None):
    if font is None:
        font = ImageFont.truetype("/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc", char_size)
    mask = font.getmask(text)
    size = mask.size[::-1]
    a = np.asarray(mask).reshape(size) / 255
    res = cv2.resize(a, dsize=(char_size, char_size), interpolation=cv2.INTER_CUBIC)
    return res

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [5]:
# https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/02-intermediate/language_model/data_utils.py
import torch
import re

class Dictionary(object):
    def __init__(self, max_size=None):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 1
        self.word2idx['⸘'] = 0 # as unk
        self.idx2word[0] = '⸘'
        self.max_size = max_size + 1
    
    def add_word(self, word):
        if not word in self.word2idx and self.idx < self.max_size:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
    
    def __len__(self):
        return len(self.word2idx)


class Corpus(object):
    def __init__(self, max_size=None):
        self.dictionary = Dictionary(max_size=max_size)

    def get_data(self, path, batch_size=20):
        # Add words to the dictionary
#         with open(path, 'r') as f:
#             tokens = 0
#             for line in f:
#                 words = line.split() + ['<eos>']
#                 tokens += len(words)
#                 for word in words: 
#                     self.dictionary.add_word(word)  

        # split words to char and add to dictionary
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                line = ' '.join(line) # split words to char
                line = re.sub(r'[" "]+', ' ', line) # remove continous space
                chars = line.split() + ['¿'] # ¿ as <eos>
                tokens += len(chars)
                for char in chars:
                    self.dictionary.add_word(char)
        
        # Tokenize the file content
        ids = torch.LongTensor(tokens)
        token = 0
        with open(path, 'r') as f:
            for line in f:
                line = ' '.join(line) # split words to char
                line = re.sub(r'[" "]+', ' ', line) # remove continous space
                chars = line.split() + ['¿'] # ¿ as <eos>
                for char in chars:
                    if char in self.dictionary.word2idx:
                        ids[token] = self.dictionary.word2idx[char]
                        token += 1
                    else:
                        ids[token] = self.dictionary.word2idx['⸘']
                        token += 1
        num_batches = ids.size(0) // batch_size
        ids = ids[:num_batches*batch_size]
        return ids.view(batch_size, -1)

In [6]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [7]:
# RNN based language model
class RNNLM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(RNNLM, self).__init__()
        self.gru = nn.GRU(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, char_cnn_o, h):
        # Embed word ids to vectors
        x = char_cnn_o
        
        # Forward propagate LSTM
        out, h = self.gru(x, h)
        
        # Reshape output to (batch_size*sequence_length, hidden_size)
        out = out.reshape(out.size(0)*out.size(1), out.size(2))
        
        # Decode hidden states of all time steps
        out = self.linear(out)
        return out, h
"""
Utility function for computing output of convolutions
takes a tuple of (h,w) and returns a tuple of (h,w)
"""
def conv_output_shape(h_w, kernel_size=1, stride=1, pad=0, dilation=1):
    from math import floor
    if type(kernel_size) is not tuple:
        kernel_size = (kernel_size, kernel_size)
    if type(stride) is not tuple:
        stride = (stride, stride)
    h = floor( ((h_w[0] + (2 * pad) - ( dilation * (kernel_size[0] - 1) ) - 1 )/ stride[0]) + 1)
    w = floor( ((h_w[1] + (2 * pad) - ( dilation * (kernel_size[1] - 1) ) - 1 )/ stride[1]) + 1)
    return h, w

# Dai et al. 's CNN glyph encoder
class Dai_CNN(nn.Module):
    def __init__(self, embed_size, input_size=(24, 24)):
        super(Dai_CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 32, (7, 7), stride=(2,2))
        torch.nn.init.xavier_uniform_(self.conv1.weight)
        torch.nn.init.zeros_(self.conv1.bias)
        h, w = conv_output_shape(input_size, (7, 7), (2, 2))
        
        self.conv2 = nn.Conv2d(32, 16, (5, 5), stride=(2,2))
        torch.nn.init.xavier_uniform_(self.conv2.weight)
        torch.nn.init.zeros_(self.conv2.bias)
        h, w = conv_output_shape((h, w), (5, 5), (2, 2))
                
        self.fc = nn.Linear(16*h*w, embed_size)
        torch.nn.init.xavier_uniform_(self.fc.weight)
        torch.nn.init.zeros_(self.fc.bias)
        
        self.h, self.w = h, w
        
    def forward(self, char_img):
        b = char_img.size(0)
        x = self.conv1(char_img)
        x = torch.nn.functional.relu(x)
        x = self.conv2(x)
        x = torch.nn.functional.relu(x)
        x = x.view(-1, 16*self.h*self.w)
        return self.fc(x)

In [8]:
# Hyper-parameters
embed_size = 300
hidden_size = 128
num_layers = 1
num_epochs = 50
batch_size = 16
seq_length = 32
learning_rate = 1e-3

# Load dataset
corpus = Corpus(max_size=4000)
ids = corpus.get_data('icwb2-data/training/msr_training.utf8', batch_size)
vocab_size = len(corpus.dictionary)
num_batches = ids.size(1) // seq_length

In [9]:
model = RNNLM(vocab_size, embed_size, hidden_size, num_layers).to(device)
cnn_encoder = Dai_CNN(embed_size, input_size=(char_size, char_size)).to(device)
model.train()
cnn_encoder.train()
params = list(model.parameters())+list(cnn_encoder.parameters())

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params, lr=learning_rate)

# Truncated backpropagation
def detach(state):
    return state.detach()

In [10]:
# Train the model
for epoch in range(num_epochs):
    # Set initial hidden and cell states
    state = torch.zeros(num_layers, batch_size, hidden_size).to(device)
    
    for i in range(0, ids.size(1) - seq_length, seq_length):
        # Get mini-batch inputs and targets
        inputs = ids[:, i:i+seq_length].to(device)
        targets = ids[:, (i+1):(i+1)+seq_length].to(device)
        
        # Get images
        images = np.zeros((inputs.size(0), inputs.size(1), char_size, char_size, 1))
        for b, seq in enumerate(ids):
            for s, idx in enumerate(ids[b][i:i+seq_length]):
                images[b, s] = np.load(f'char_img/noto_CJK/msr3/{idx}.npy').reshape(char_size,char_size,1)
        images = torch.from_numpy(images).float().to(device) # B N H W C
        images = images.view(-1, char_size, char_size, 1) # B*N H W C
        images = images.permute(0, 3, 1, 2) # from B*N H W C to B*N C H W
        
        # Get encoded images
        cnn_o = cnn_encoder(images)
        cnn_o = cnn_o.view(inputs.size(0), inputs.size(1), -1)
               
        # Forward pass
        state = detach(state)
        outputs, state = model(cnn_o, state)
        loss = criterion(outputs, targets.reshape(-1))
        
        # Backward and optimize
        model.zero_grad()
        cnn_encoder.zero_grad()
        loss.backward()
        clip_grad_norm_(params, 0.5)
        optimizer.step()

        step = (i+1) // seq_length
        if step % 1000 == 0:
            print ('{} Epoch [{}/{}], Step[{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'
                   .format(datetime.now(), epoch+1, num_epochs, step, num_batches, loss.item(), np.exp(loss.item())))

2019-11-25 17:09:00.825180 Epoch [1/50], Step[0/8080], Loss: 8.2959, Perplexity: 4007.45
2019-11-25 17:15:20.302716 Epoch [1/50], Step[1000/8080], Loss: 5.9313, Perplexity: 376.66
2019-11-25 17:21:37.509303 Epoch [1/50], Step[2000/8080], Loss: 5.3139, Perplexity: 203.14
2019-11-25 17:27:59.752441 Epoch [1/50], Step[3000/8080], Loss: 4.9082, Perplexity: 135.40
2019-11-25 17:34:25.121433 Epoch [1/50], Step[4000/8080], Loss: 4.8073, Perplexity: 122.40
2019-11-25 17:41:48.158266 Epoch [1/50], Step[5000/8080], Loss: 5.1813, Perplexity: 177.91
2019-11-25 17:48:54.614945 Epoch [1/50], Step[6000/8080], Loss: 5.0580, Perplexity: 157.28
2019-11-25 17:55:51.318983 Epoch [1/50], Step[7000/8080], Loss: 4.6831, Perplexity: 108.10
2019-11-25 18:02:56.155131 Epoch [1/50], Step[8000/8080], Loss: 4.6707, Perplexity: 106.77
2019-11-25 18:03:31.046830 Epoch [2/50], Step[0/8080], Loss: 4.6619, Perplexity: 105.84
2019-11-25 18:10:34.486079 Epoch [2/50], Step[1000/8080], Loss: 4.6841, Perplexity: 108.21
2019

2019-11-26 02:14:21.784382 Epoch [11/50], Step[2000/8080], Loss: 4.2067, Perplexity: 67.13
2019-11-26 02:21:03.811082 Epoch [11/50], Step[3000/8080], Loss: 4.1389, Perplexity: 62.73
2019-11-26 02:27:49.800121 Epoch [11/50], Step[4000/8080], Loss: 4.0466, Perplexity: 57.20
2019-11-26 02:34:32.614196 Epoch [11/50], Step[5000/8080], Loss: 4.5631, Perplexity: 95.88
2019-11-26 02:41:12.030792 Epoch [11/50], Step[6000/8080], Loss: 4.3595, Perplexity: 78.22
2019-11-26 02:47:57.500543 Epoch [11/50], Step[7000/8080], Loss: 3.9911, Perplexity: 54.11
2019-11-26 02:54:41.872821 Epoch [11/50], Step[8000/8080], Loss: 4.2026, Perplexity: 66.86
2019-11-26 02:55:14.460793 Epoch [12/50], Step[0/8080], Loss: 4.2834, Perplexity: 72.48
2019-11-26 03:02:00.700610 Epoch [12/50], Step[1000/8080], Loss: 4.1152, Perplexity: 61.26
2019-11-26 03:08:44.149891 Epoch [12/50], Step[2000/8080], Loss: 4.1998, Perplexity: 66.67
2019-11-26 03:15:27.755647 Epoch [12/50], Step[3000/8080], Loss: 4.1336, Perplexity: 62.40
20

2019-11-26 11:23:35.953235 Epoch [21/50], Step[3000/8080], Loss: 4.0283, Perplexity: 56.17
2019-11-26 11:30:20.166043 Epoch [21/50], Step[4000/8080], Loss: 4.0650, Perplexity: 58.26
2019-11-26 11:37:04.211133 Epoch [21/50], Step[5000/8080], Loss: 4.5362, Perplexity: 93.33
2019-11-26 11:43:46.354274 Epoch [21/50], Step[6000/8080], Loss: 4.3013, Perplexity: 73.80
2019-11-26 11:50:29.287913 Epoch [21/50], Step[7000/8080], Loss: 3.9532, Perplexity: 52.10
2019-11-26 11:57:17.300156 Epoch [21/50], Step[8000/8080], Loss: 4.1866, Perplexity: 65.80
2019-11-26 11:57:49.604476 Epoch [22/50], Step[0/8080], Loss: 4.2463, Perplexity: 69.84
2019-11-26 12:04:33.883542 Epoch [22/50], Step[1000/8080], Loss: 4.0545, Perplexity: 57.66
2019-11-26 12:11:15.182698 Epoch [22/50], Step[2000/8080], Loss: 4.1821, Perplexity: 65.50
2019-11-26 12:17:55.466937 Epoch [22/50], Step[3000/8080], Loss: 4.0097, Perplexity: 55.13
2019-11-26 12:24:41.053405 Epoch [22/50], Step[4000/8080], Loss: 4.0620, Perplexity: 58.09
20

2019-11-26 20:38:43.573941 Epoch [31/50], Step[4000/8080], Loss: 4.0750, Perplexity: 58.85
2019-11-26 20:45:33.588903 Epoch [31/50], Step[5000/8080], Loss: 4.5274, Perplexity: 92.52
2019-11-26 20:52:21.471447 Epoch [31/50], Step[6000/8080], Loss: 4.2727, Perplexity: 71.72
2019-11-26 20:59:09.389796 Epoch [31/50], Step[7000/8080], Loss: 3.9272, Perplexity: 50.77
2019-11-26 21:06:00.595902 Epoch [31/50], Step[8000/8080], Loss: 4.1320, Perplexity: 62.30
2019-11-26 21:06:34.746504 Epoch [32/50], Step[0/8080], Loss: 4.2026, Perplexity: 66.86
2019-11-26 21:13:23.622938 Epoch [32/50], Step[1000/8080], Loss: 4.0166, Perplexity: 55.51
2019-11-26 21:20:13.487050 Epoch [32/50], Step[2000/8080], Loss: 4.1691, Perplexity: 64.66
2019-11-26 21:27:01.936793 Epoch [32/50], Step[3000/8080], Loss: 3.9979, Perplexity: 54.48
2019-11-26 21:33:52.961344 Epoch [32/50], Step[4000/8080], Loss: 4.0505, Perplexity: 57.43
2019-11-26 21:40:45.689071 Epoch [32/50], Step[5000/8080], Loss: 4.5245, Perplexity: 92.25
20

2019-11-27 05:57:11.921065 Epoch [41/50], Step[5000/8080], Loss: 4.5085, Perplexity: 90.78
2019-11-27 06:03:53.958136 Epoch [41/50], Step[6000/8080], Loss: 4.2842, Perplexity: 72.54
2019-11-27 06:10:32.121488 Epoch [41/50], Step[7000/8080], Loss: 3.9565, Perplexity: 52.27
2019-11-27 06:17:11.585669 Epoch [41/50], Step[8000/8080], Loss: 4.1118, Perplexity: 61.06
2019-11-27 06:17:43.752654 Epoch [42/50], Step[0/8080], Loss: 4.1660, Perplexity: 64.45
2019-11-27 06:24:27.363837 Epoch [42/50], Step[1000/8080], Loss: 4.0776, Perplexity: 59.00
2019-11-27 06:31:03.391964 Epoch [42/50], Step[2000/8080], Loss: 4.1632, Perplexity: 64.28
2019-11-27 06:37:43.928041 Epoch [42/50], Step[3000/8080], Loss: 3.9776, Perplexity: 53.39
2019-11-27 06:44:24.703259 Epoch [42/50], Step[4000/8080], Loss: 4.0352, Perplexity: 56.56
2019-11-27 06:51:04.587880 Epoch [42/50], Step[5000/8080], Loss: 4.5281, Perplexity: 92.58
2019-11-27 06:57:46.880454 Epoch [42/50], Step[6000/8080], Loss: 4.2589, Perplexity: 70.73
20

In [11]:
model.eval()
cnn_encoder.eval()

perplexity = .0
num_step = 0
for i in range(0, ids.size(1) - seq_length, seq_length):
    # Get mini-batch inputs and targets
    inputs = ids[:, i:i+seq_length].to(device)
    targets = ids[:, (i+1):(i+1)+seq_length].to(device)

    # Get images
    images = np.zeros((inputs.size(0), inputs.size(1), char_size, char_size, 1))
    for b, seq in enumerate(ids):
        for s, idx in enumerate(ids[b][i:i+seq_length]):
            images[b, s] = np.load(f'char_img/noto_CJK/msr3/{idx}.npy').reshape(char_size,char_size,1)
    images = torch.from_numpy(images).float().to(device) # B N H W C
    images = images.view(-1, char_size, char_size, 1) # B*N H W C
    images = images.permute(0, 3, 1, 2) # from B*N H W C to B*N C H W

    # Get encoded images
    cnn_o = cnn_encoder(images)
    cnn_o = torch.reshape(cnn_o, (inputs.size(0), inputs.size(1), -1))

    # Forward pass
    state = detach(state)
    outputs, state = model(cnn_o, state)
    loss = criterion(outputs, targets.reshape(-1))
    
    perplexity += np.exp(loss.item())

    num_step += 1
    
print(f"Train Perplexity: {perplexity / num_step}")

Train Perplexity: 64.87206335331925


In [12]:
test_ids = corpus.get_data('icwb2-data/testing/msr_test.utf8', batch_size)
# filter out unknown character
test_ids = test_ids.view(-1)
mask = test_ids < vocab_size
test_ids = test_ids[mask]
num_batches = test_ids.size(0) // batch_size
test_ids = test_ids[:num_batches*batch_size]
test_ids = test_ids.view(batch_size, -1)

In [13]:
perplexity = .0
num_step = 0
for i in range(0, test_ids.size(1) - seq_length, seq_length):
    # Get mini-batch inputs and targets
    inputs = test_ids[:, i:i+seq_length].to(device)
    targets = test_ids[:, (i+1):(i+1)+seq_length].to(device)

    # Get images
    images = np.zeros((inputs.size(0), inputs.size(1), char_size, char_size, 1))
    for b, seq in enumerate(test_ids):
        for s, idx in enumerate(test_ids[b][i:i+seq_length]):
            images[b, s] = np.load(f'char_img/noto_CJK/msr3/{idx}.npy').reshape(char_size,char_size,1)
    images = torch.from_numpy(images).float().to(device) # B N H W C
    images = images.view(-1, char_size, char_size, 1) # B*N H W C
    images = images.permute(0, 3, 1, 2) # from B*N H W C to B*N C H W

    # Get encoded images
    cnn_o = cnn_encoder(images)
    cnn_o = torch.reshape(cnn_o, (inputs.size(0), inputs.size(1), -1))

    # Forward pass
    state = detach(state)
    outputs, state = model(cnn_o, state)
    loss = criterion(outputs, targets.reshape(-1))

    perplexity += np.exp(loss.item())

    num_step += 1
    
print(f"Test Perplexity: {perplexity / num_step}")

Test Perplexity: 70.38195446893054
